In [29]:
import math
import librosa
import itertools
import numpy as np
import seaborn as sns
from scipy.io import wavfile
import IPython.display as ipd
import matplotlib.pyplot as plt

sns.set_theme()
SR = 44_100 # Sample rate

figsize=(25, 6.25)
colors = "#323031", "#308E91", "#34369D","#5E2A7E", "#5E2A7E", "#6F3384"

In [119]:
hz = lambda note:librosa.note_to_hz(note)
getfig = lambda : plt.figure(figsize=figsize)
savefig = lambda fig, name: fig.savefig(f"./tempimg/{name}.jpg")
to_16 = lambda wav, amp: np.int16(wav * amp * (2**15 - 1))

def plot(xy, r=1,c=1,i=1,title="", xlabel="",ylabel="",yticks=None, xticks=None,**plot_kwargs):
    # plt.plot helper
    if r > 0:
        plt.subplot(r,c,i)
    plt.title(title)
    if len(xy) == 2:
        plt.plot(*xy, **plot_kwargs)
    else:
        plt.plot(xy, **plot_kwargs)
        
    if xticks is not None: plt.xticks(xticks)
    if yticks is not None: plt.yticks(yticks)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    
def getval(osc, count=SR, it=False):
    if it: osc = iter(osc)
    # returns 1 sec of samples of given osc.
    return [next(osc) for i in range(count)]

def getseq(osc, notes=["C4", "E4", "G4"], note_lens=[0.5,0.5,0.5]):
    # Returns samples of the note seq for the given osc.
    samples = []
    osc = iter(osc)
    for note, note_len in zip(notes, note_lens):
        osc.freq = librosa.note_to_hz(note)
        for _ in range(int(SR * note_len)):
            samples.append(next(osc))
    return samples

def wave_to_file(wav, wav2=None, fname="temp", amp=0.1):
    wav = np.array(wav)
    wav = to_16(wav, amp)
    if wav2 is not None:
        wav2 = np.array(wav2)
        wav2 = to_16(wav2, amp)
        wav = np.stack([wav, wav2]).T
    
    wavfile.write(f"tempsnd/{fname}.wav", SR, wav)
    
def fplot_xy(wave, fslice=slice(0,100), sample_rate=SR):
    # Returns FFTed samples of input wave
    fd = np.fft.fft(wave)
    fd_mag = np.abs(fd)
    x = np.linspace(0, sample_rate, len(wave))
    y = fd_mag * 2 / sample_rate
    return x[fslice], y[fslice]


In [1]:
from abc import ABC, abstractmethod

class Oscillator(ABC):
    def __init__(self, freq=440, phase=0, amp=1, \
                 sample_rate=44_100, wave_range=(-1, 1)):
        self._freq = freq
        self._amp = amp
        self._phase = phase
        self._sample_rate = sample_rate
        self._wave_range = wave_range
        
        # Properties that will be changed
        self._f = freq
        self._a = amp
        self._p = phase
        
    @property
    def init_freq(self):
        return self._freq
    
    @property
    def init_amp(self):
        return self._amp
    
    @property
    def init_phase(self):
        return self._phase
    
    @property
    def freq(self):
        return self._f
    
    @freq.setter
    def freq(self, value):
        self._f = value
        self._post_freq_set()
        
    @property
    def amp(self):
        return self._a
    
    @amp.setter
    def amp(self, value):
        self._a = value
        self._post_amp_set()
        
    @property
    def phase(self):
        return self._p
    
    @phase.setter
    def phase(self, value):
        self._p = value
        self._post_phase_set()
    
    def _post_freq_set(self):
        pass
    
    def _post_amp_set(self):
        pass
    
    def _post_phase_set(self):
        pass
    
    @abstractmethod
    def _initialize_osc(self):
        pass
    
    @staticmethod
    def squish_val(val, min_val=0, max_val=1):
        return (((val + 1) / 2 ) * (max_val - min_val)) + min_val
    
    @abstractmethod
    def __next__(self):
        return None
    
    def __iter__(self):
        self.freq = self._freq
        self.phase = self._phase
        self.amp = self._amp
        self._initialize_osc()
        return self

In [275]:
class SineOscillator(Oscillator):
    def _post_freq_set(self):
        self._step = (2 * math.pi * self._f) / self._sample_rate
        
    def _post_phase_set(self):
        self._p = (self._p / 360) * 2 * math.pi
        
    def _initialize_osc(self):
        self._i = 0
        
    def __next__(self):
        val = math.sin(self._i + self._p)
        self._i = self._i + self._step
        if self._wave_range is not (-1, 1):
            val = self.squish_val(val, *self._wave_range)
        return val * self._a

<>:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/9k/_vz3shh92wq2gs0dz81qxnjm0000gn/T/ipykernel_25685/1144164654.py:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self._wave_range is not (-1, 1):


In [3]:
class SquareOscillator(SineOscillator):
    def __init__(self, freq=440, phase=0, amp=1, \
                 sample_rate=44_100, wave_range=(-1, 1), threshold=0):
        super().__init__(freq, phase, amp, sample_rate, wave_range)
        self.threshold = threshold
    
    def __next__(self):
        val = math.sin(self._i + self._p)
        self._i = self._i + self._step
        if val < self.threshold:
            val = self._wave_range[0]
        else:
            val = self._wave_range[1]
        return val * self._a

In [4]:
class SawtoothOscillator(Oscillator):
    def _post_freq_set(self):
        self._period = self._sample_rate / self._f
        self._post_phase_set
        
    def _post_phase_set(self):
        self._p = ((self._p + 90)/ 360) * self._period
    
    def _initialize_osc(self):
        self._i = 0
    
    def __next__(self):
        div = (self._i + self._p )/self._period
        val = 2 * (div - math.floor(0.5 + div))
        self._i = self._i + 1
        if self._wave_range is not (-1, 1):
            val = self.squish_val(val, *self._wave_range)
        return val * self._a

<>:16: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:16: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/9k/_vz3shh92wq2gs0dz81qxnjm0000gn/T/ipykernel_25685/332260643.py:16: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self._wave_range is not (-1, 1):


In [5]:
class TriangleOscillator(SawtoothOscillator):
    def __next__(self):
        div = (self._i + self._p)/self._period
        val = 2 * (div - math.floor(0.5 + div))
        val = (abs(val) - 0.5) * 2
        self._i = self._i + 1
        if self._wave_range is not (-1, 1):
            val = self.squish_val(val, *self._wave_range)
        return val * self._a

<>:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/9k/_vz3shh92wq2gs0dz81qxnjm0000gn/T/ipykernel_25685/29090328.py:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self._wave_range is not (-1, 1):


In [276]:
class ADSREnvelope:
    def __init__(self, attack_duration=0.05, decay_duration=0.2, sustain_level=0.7, \
                 release_duration=0.3, sample_rate=44100):
        self.attack_duration = attack_duration
        self.decay_duration = decay_duration
        self.sustain_level = sustain_level
        self.release_duration = release_duration
        self._sample_rate = sample_rate
        
    def get_ads_stepper(self):
        steppers = []
        if self.attack_duration > 0:
            steppers.append(itertools.count(start=0, \
                step= 1 / (self.attack_duration * self._sample_rate)))
        if self.decay_duration > 0:
            steppers.append(itertools.count(start=1, \
                step=-(1 - self.sustain_level) / (self.decay_duration  * self._sample_rate)))
        while True:
            l = len(steppers)
            if l > 0:
                val = next(steppers[0])
                if l == 2 and val > 1:
                    steppers.pop(0)
                    val = next(steppers[0])
                elif l == 1 and val < self.sustain_level:
                    steppers.pop(0)
                    val = self.sustain_level
            else:
                val = self.sustain_level
            yield val
    
    def get_r_stepper(self):
        val = 1
        if self.release_duration > 0:
            release_step = - self.val / (self.release_duration * self._sample_rate)
            stepper = itertools.count(self.val, step=release_step)
        else:
            val = -1
        while True:
            if val <= 0:
                self.ended = True
                val = 0
            else:
                val = next(stepper)
            yield val
    
    def __iter__(self):
        self.val = 0
        self.ended = False
        self.stepper = self.get_ads_stepper()
        return self
    
    def __next__(self):
        self.val = next(self.stepper)
        return self.val
        
    def trigger_release(self):
        self.stepper = self.get_r_stepper()

In [410]:
# write a frequency modulator class
class FrequencyModulator:
    """need to have smth like CEGEC CEGEC"""
    def __init__(self, notes=["C4", "E4", "G4"], note_lens=[0.5,0.5,0.5], duration=1.5, sample_rate=44100):
        self.notes = notes
        self.note_lens = note_lens
        self.duration = duration
        self._sample_rate = sample_rate
        self.counter = 0
        
        length_flag = sum(self.note_lens)< self.duration
        while length_flag:
            for i in range(len(note_lens)):
                self.notes.append(notes[i])
                self.note_lens.append(note_lens[i])
                if sum(self.note_lens)> self.duration:
                    self.note_lens[-2] = self.note_lens[-2] - sum(self.note_lens) + self.duration
                    length_flag = False
                    break
        self.note_steps = [int(length*self._sample_rate) for length in self.note_lens]
        self.total_steps = int(self._sample_rate * self.duration)
        if sum(self.note_steps) != self.total_steps:
                self.note_steps[-1] -=sum(self.note_steps) - self.total_steps
        print(self.note_steps, sum(self.note_steps), self.total_steps)
        assert sum(self.note_steps) == self.total_steps
    
    def get_freq(self): # list value you need
        self.val = 0
        try:
            self.note_steps[0]
        except IndexError:
            val = 0
            self.ended = True
            return val
        while self.counter <= self.note_steps[0]:
            if self.counter < self.note_steps[0]:
                val = self.notes[0]
                self.counter+=1
            else:
                self.note_steps.pop(0)
                self.notes.pop(0)
                try:
                    val = self.notes[0]
                except IndexError:
                    self.ended = True
                    break
                self.counter = 0
            return val
        val = 0
        return val

    def __iter__(self): # get the value
        self.val = 0
        self.ended = False
        return self
    
    def __next__(self): # get the value
        self.val = self.get_freq()
        return self.val

In [162]:
fm = FrequencyModulator()
iter(fm)
next(fm)

'C4'

In [278]:
class ModulatedOscillator:
    def __init__(self, oscillator, *modulators, amp_mod=None, freq_mod=None, phase_mod=None):
        self.oscillator = oscillator
        self.modulators = modulators # list
        self.amp_mod = amp_mod
        self.freq_mod = freq_mod
        self.phase_mod = phase_mod
        self._modulators_count = len(modulators)
    
    def __iter__(self):
        iter(self.oscillator)
        [iter(modulator) for modulator in self.modulators]
        return self
    
    def _modulate(self, mod_vals):
        if self.amp_mod is not None:
            new_amp = self.amp_mod(self.oscillator.init_amp, mod_vals[0])
            self.oscillator.amp = new_amp
        if self.freq_mod is not None:
            if self._modulators_count == 2:
                mod_val = mod_vals[1]
            else:
                mod_val = mod_vals[0]
            new_freq = self.freq_mod(self.oscillator.init_freq, mod_val)
            self.oscillator.freq = new_freq
            
        if self.phase_mod is not None:
            if self._modulators_count == 3:
                mod_val = mod_vals[2]
            else:
                mod_val = mod_vals[-1]
            new_phase = self.phase_mod(self.oscillator.init_phase, mod_val)
            self.oscillator.phase = new_phase
    
    def trigger_release(self):
        tr = "trigger_release"
        for modulator in self.modulators:
            if hasattr(modulator, tr):
                modulator.trigger_release()
        if hasattr(self.oscillator, tr):
            self.oscillator.trigger_release()
            
    @property
    def ended(self):
        e = "ended"
        ended = []
        for modulator in self.modulators:
            if hasattr(modulator, e):
                ended.append(modulator.ended)
        if hasattr(self.oscillator, e):
            ended.append(self.oscillator.ended)
        return all(ended)

    def __next__(self):
        mod_vals = [next(modulator) for modulator in self.modulators]
        self._modulate(mod_vals)
        return next(self.oscillator)

In [279]:
def amp_mod(init_amp, env):
    return env * init_amp
    
def freq_mod(init_freq, env, mod_amt=0.01, sustain_level=0.7):
    return init_freq + ((env - sustain_level) * init_freq * mod_amt)

def getdownlen(env, suslen, sample_rate=SR):
    n = sum(env.attack_duration, env.release_duration, suslen)
    return int(n * sample_rate)

def gettrig(gen, downtime, sample_rate=SR):
    gen = iter(gen)
    down = int(downtime * sample_rate)
    vals = getval(gen, down)
    gen.trigger_release()
    while not gen.ended:
        vals.append(next(gen))
    return vals

def getadsr(a, d, sl, sd, r, Osc=SquareOscillator(55), mod = None):
    if mod is None:
        mod = ModulatedOscillator(
            Osc,
            ADSREnvelope(a,d,sl,r),
            amp_mod=amp_mod
        )
    downtime = a + d + sd
    return gettrig(mod, downtime)

def getseq(osc, notes=["C4", "E4", "G4"], note_lens=[0.5,0.5,0.5]):
    # Returns samples of the note seq for the given osc.
    samples = []
    osc = iter(osc)
    for note, note_len in zip(notes, note_lens):
        osc.freq = librosa.note_to_hz(note)
        for _ in range(int(SR * note_len)):
            samples.append(next(osc))
    return samples

def freq_mod_2(init_freq, env):
    return env

In [ ]:
new_freq = self.freq_mod(self.oscillator.init_freq, mod_val)

In [45]:
fig = getfig()
vals = getadsr(0.02, 0.75, 0, 0, 0)
# plot(vals,1,3,1, title="Droppers", xlabel="samples", ylabel="amplitude", color=colors[2])

<Figure size 2500x625 with 0 Axes>

In [46]:
wave_to_file(vals, fname="55Hz_freq_and_amp_env")

In [48]:
ipd.Audio("./tempsnd/55Hz_freq_and_amp_env.wav")

In [245]:
class Chain:
    def __init__(self, generator, *modifiers):
        self.generator = generator
        self.modifiers = modifiers
        
    def __getattr__(self, attr):
        val = None
        if hasattr(self.generator, attr):
            val = getattr(self.generator, attr)
        else:
            for modifier in self.modifiers:
                if hasattr(modifier, attr):
                    val = getattr(modifier, attr)
                    break
            else:
                raise AttributeError(f"attribute '{attr}' does not exist")
        return val
    
    def trigger_release(self):
        tr = "trigger_release"
        if hasattr(self.generator, tr):
            self.generator.trigger_release()
        for modifier in self.modifiers:
            if hasattr(modifier, tr):
                modifier.trigger_release()
                
    @property
    def ended(self):
        ended = []; e = "ended"
        if hasattr(self.generator, e):
            ended.append(self.generator.ended)
        ended.extend([m.ended for m in self.modifiers if hasattr(m, e)])
        return all(ended)
    
    def __iter__(self):
        iter(self.generator)
        [iter(mod) for mod in self.modifiers if hasattr(mod, "__iter__")]
        return self
        
    def __next__(self):
        val = next(self.generator)
        [next(mod) for mod in self.modifiers if hasattr(mod, "__iter__")]
        for modifier in self.modifiers:
            val = modifier(val)
        return val

In [280]:
from collections.abc import Iterable

class WaveAdder:
    def __init__(self, *generators, stereo=False):
        self.generators = generators
        self.stereo = stereo
        
    def _mod_channels(self, _val):
        val = _val
        if isinstance(_val, (int, float)) and self.stereo:
            val = (_val, _val)
        elif isinstance(_val, Iterable) and not self.stereo:
            val = sum(_val)/len(_val)
        return val
    
    def trigger_release(self):
        [gen.trigger_release() for gen in self.generators if hasattr(gen, "trigger_release")]
    
    @property
    def ended(self):
        ended = [gen.ended for gen in self.generators if hasattr(gen, "ended")]
        return all(ended)
    
    def __iter__(self):
        [iter(gen) for gen in self.generators]
        return self
            
    def __next__(self):
        vals = [self._mod_channels(next(gen)) for gen in self.generators]
        if self.stereo:
            l, r = zip(*vals)
            val = (sum(l)/len(l), sum(r)/len(r))
        else:
            val = sum(vals)/ len(vals)
        return val

### Data generation process

In [97]:
def get_octive(freq, freq_range):
    octives = [2, 3, 4]
    octive_list = []
    octive_up = [freq*e for e in octives]
    octive_down = [freq/e for e in octives]
    octive_list = [freq] + octive_up + octive_down
    octive_list = [e for e in octive_list if e > freq_range[0] and e < freq_range[1]]
    return octive_list

In [165]:
print(hz("A4"), hz("C4"), hz("E4"))
class ADSREnvelope:
    def __init__(self, attack_duration=0.05, decay_duration=0.2, sustain_level=0.7, \
                 release_duration=0.3, sample_rate=44100):
class FrequencyModulator:
    """need to have smth like CEGEC CEGEC"""
    def __init__(self, notes=["C4", "E4", "G4"], note_lens=[0.5,0.5,0.5], duration=1.5, sample_rate=44100)

440.0 261.6255653005986 329.6275569128699


In [317]:
dur = 1.5
gen = WaveAdder(
    ModulatedOscillator(
        SineOscillator(hz("A4")),
        ADSREnvelope(0.01, 0.1, 0.8, 0.01),
        FrequencyModulator(notes=[440.0, 261.6255653005986, 329.6275569128699]),
        amp_mod=amp_mod,
        freq_mod = freq_mod_2
    ),
    ModulatedOscillator(
        SineOscillator(hz("A4")),
        ADSREnvelope(0.01, 0.1, 0.8, 0.01),
        FrequencyModulator(notes=[440.0/2], note_lens = [1.5]),
        amp_mod=amp_mod,
        freq_mod = freq_mod_2
    ),
    
    stereo=False
)

[440.0, 261.6255653005986, 329.6275569128699]
[220.0]


In [318]:
wav = gettrig(gen, 0.01+0.1-0.01+1.5)
wave_to_file(wav, fname="test1")
ipd.Audio("tempsnd/test1.wav")

In [216]:
print(1.5*44100, 0.5*44100)

66150.0 22050.0


In [412]:
# data generation process
FREQ_RANGE = (130.81, 523.25) #c2 to c5
NB_SAMPLE = 10
CHORD = "major" # major or minor
CHORD_TYPE = "triad" # triad or seventh

# sample one frequency
freq = np.random.uniform(FREQ_RANGE[0], FREQ_RANGE[1])
print(freq)

# get notes for chord
first_note = freq
second_note = freq*pow(2, 2/12)
third_note = freq*pow(2, 4/12)
fourth_note = freq*pow(2, 5/12)
fifth_note = freq*pow(2, 7/12)
sixth_note = freq*pow(2, 9/12)
seventh_note = freq*pow(2, 11/12)
first_list = get_octive(first_note, FREQ_RANGE)
third_list = get_octive(third_note, FREQ_RANGE)
fifth_list = get_octive(fifth_note, FREQ_RANGE)

scale = np.array([first_note, second_note, third_note, fourth_note, fifth_note, sixth_note, seventh_note])
sampled_index = np.random.randint(0, len(scale)-1,4)
sampled_notes = list(scale[sampled_index])
sampled_notes.append(first_note)
sampled_time = list(np.random.uniform(0.3, 1, 5))
# sampled_time.append(4.0-sum(sampled_time))
print(scale)
print(sampled_index)
print((sampled_time))


gen = WaveAdder(
    ModulatedOscillator(
        SineOscillator(),
        ADSREnvelope(0.01, 0.1, 0.8, 0.01),
        FrequencyModulator(notes=sampled_notes, note_lens=sampled_time, duration=4.0),
        amp_mod=amp_mod,
        freq_mod = freq_mod_2
    ),
    ModulatedOscillator(
        SineOscillator(),
        ADSREnvelope(0.01, 0.1, 0.8, 0.01),
        FrequencyModulator(notes=[freq], note_lens=[4.0], duration=4.0),
        amp_mod=amp_mod,
        freq_mod = freq_mod_2
    ),
    
    stereo=False
)

wav = gettrig(gen, 0.01+0.1-0.01+4)
wave_to_file(wav, fname="test2")
ipd.Audio("tempsnd/test2.wav")
# sample nb of oscillator [3,5]

# sample one from first_list
# sample one from third_list
# sample one from fifth_list

# sample nb additional notes



257.96341325548667
[257.96341326 289.55414123 325.01353446 344.33984493 386.50840766
 433.84101895 486.97007877]
[4 1 1 5]
[0.30093580470115044, 0.6515786209963911, 0.47130268836413597, 0.5688493993851174, 0.7613628699517168]
[13271, 28734, 20784, 25086, 33576, 13271, 20891, 20787] 176400 176400
[176400] 176400 176400
